In [1]:
import numpy as np
from astropy.io import fits
from skipper import observe

### import pre-made tiliing file

In [2]:
hdul2 = fits.open('../data/decam-tiles-decals-merian.fits.gz')
data2 = hdul2[1].data
tileID2 = data2['TILEID']
pass_num2 = data2['PASS']
ra2 = data2['RA']
dec2 = data2['DEC']

#### initialize ObservingCatalog; build JSON from first 100 entries

In [3]:
ocat = observe.ObsCatalog (comment='TEST')

In [4]:
df = ocat.build_catalog ( ra2[:100], dec2[:100], 'science', 'N708', 'science', 15.*60 )
ocat.to_json ( df )

In [5]:
df

,comment,filter,seqtot,seqnum,expType,object,proposer,program,RA,propid,dec,expTime
0,TEST,N708,100,1,science,science,Leauthaud,Merian,239.125,2020B-XXXX,2.590,900.0
1,TEST,N708,100,2,science,science,Leauthaud,Merian,237.849,2020B-XXXX,1.333,900.0
2,TEST,N708,100,3,science,science,Leauthaud,Merian,233.860,2020B-XXXX,3.207,900.0
3,TEST,N708,100,4,science,science,Leauthaud,Merian,238.655,2020B-XXXX,4.131,900.0
4,TEST,N708,100,5,science,science,Leauthaud,Merian,237.344,2020B-XXXX,2.885,900.0
5,TEST,N708,100,6,science,science,Leauthaud,Merian,231.634,2020B-XXXX,4.808,900.0
6,TEST,N708,100,7,science,science,Leauthaud,Merian,239.503,2020B-XXXX,6.758,900.0
7,TEST,N708,100,8,science,science,Leauthaud,Merian,238.170,2020B-XXXX,5.640,900.0
8,TEST,N708,100,9,science,science,Leauthaud,Merian,236.852,2020B-XXXX,4.395,900.0
9,TEST,N708,100,10,science,science,Leauthaud,Merian,235.551,2020B-XXXX,3.122,900.0
